In [16]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True
)

model = AutoModelForCausalLM.from_pretrained(
    "openai-community/gpt2-xl",
    quantization_config=quantization_config,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2-xl")
inputs = tokenizer("Once upon a time, there was a magical forest", return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

ValueError: Using a `device_map`, `tp_plan`, `torch.device` context manager or setting `torch.set_default_device(device)` requires `accelerate`. You can install it with `pip install accelerate`

In [ ]:
#CONFIGURACIÓN DEL MODELO 

from transformers import GPT2Config, GPT2LMHeadModel, AutoTokenizer
import torch, math, json

tok = AutoTokenizer.from_pretrained("usuario/smiles-tokenizer",
                                    padding_side="right")
tok.add_special_tokens({"pad_token": "[PAD]"})   # si no lo trae

max_len = 256          # longitud SMILES 99‑percentil en tu dataset
d_model = 768          # → 12 cabezas de 64 d cada una

config = GPT2Config(   #parámetros
    vocab_size      = tok.vocab_size,
    n_positions     = max_len,          # Es la longitud máxima de secuencia con la q se podria usar el modelo 
    n_ctx           = max_len,          # alias conservado por backward‑compat
    n_embd          = d_model,          
    n_layer         = 12,               # profundidad
    n_head          = 12,               # d_model/n_head debe ser entero
    n_inner         = 4*d_model,        # feed‑forward (por defecto)
    activation_function = "gelu_new",
    resid_pdrop     = 0.1, 
    embd_pdrop      = 0.1,
    attn_pdrop      = 0.1,
    layer_norm_epsilon = 1e-5,
    bos_token_id    = tok.bos_token_id,
    eos_token_id    = tok.eos_token_id,
    pad_token_id    = tok.pad_token_id, # importante para loss masking
    use_cache       = True              # ↑ útil en generación
)

model = GPT2LMHeadModel(config)
model.resize_token_embeddings(tok.vocab_size)    # por si el tokenizer creció


In [ ]:
# pagina hugging face:  https://huggingface.co/docs/transformers/model_doc/gpt2


model.save_pretrained("./gpt2-smiles")
tokenizer.save_pretrained("./gpt2-smiles")  #Guardo el modelo y tokenizer

# terminal: echo -e "Hello, I'm a language model" | transformers run --task text-generation --model openai-community/gpt2 --device 0



NameError: name 'model' is not defined

In [ ]:
# CUANTIFICAR EL MODELO PARA REDUCIR CARGA DE MEMORIA BY PESOS DE MENOR PRESICION 